## Embeddings

Similar words occur in similar contexts. This idea is used to map words to small vectors called *embeddings* which are going to be close to each other when words have similar meanings and far apart when they don't.

### Word2Vec

1. For each word, map it to an embedding, initially a random one.
2. Use the embedding to predict the context of a word. The context is simply the words that are nearby (in a window around the original word).
3. The model used to predict a nearby word is a logistic regression.
4. Use tSNE to visually examine the resulting word distance.

<img src="https://raw.githubusercontent.com/Runze/ud730-deep-learning-class-notes/master/screenshots/lesson-5-word2vec.png" alt="alt text" width="700">

To compare embeddings, use cosine distance:
$${\mathbf {A} \cdot \mathbf {B}  \over \|\mathbf {A} \|_{2}\|\mathbf {B} \|_{2}}={\frac {\sum \limits _{i=1}^{n}{A_{i}B_{i}}}{{\sqrt {\sum \limits _{i=1}^{n}{A_{i}^{2}}}}{\sqrt {\sum \limits _{i=1}^{n}{B_{i}^{2}}}}}}$$